In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.environ.get('PATH_CUSTOM_MODULES'))

import modeling

### Prepare all basic variable

In [2]:
path_result = os.environ.get('PATH_TEST_RESULT')
path_storage = os.environ.get('PATH_EVALUATION_RESULT')

### Getting the result data

In [3]:
result = modeling.merge_result_data(path_src=path_result)

In [4]:
result.to_csv(os.path.join(path_storage,
                            'merge_result.csv'),
                index=False)

### Get the final result from cross validation

In [5]:
result.drop(columns=['fold'], inplace=True)
result_final = result.groupby(by=['scenario', 'model', 'dataset']).agg('mean')

### Get the F1-score

In [6]:
result_final['f1_score'] = 2*((result_final['precision'] * result_final['sensitivity'])
                                / (result_final['precision'] + result_final['sensitivity']))

In [7]:
result_final.reset_index(level=[0,1,2], inplace=True)

In [8]:
result_final.to_csv(os.path.join(path_storage,
                                'merge_result_final.csv'),
                    index=False)

### Analyze the result

#### By scenario

##### Scenario 1

In [9]:
# the general result
s1_result = result_final.groupby(by='scenario').get_group('scenario_1')

s1_result.drop(columns=['scenario', 'loss', 'sensitivity'], inplace=True)
s1_result.sort_values(by=['auc', 'f1_score', 'accuracy', 'precision'], ascending=False, inplace=True)
s1_result = s1_result.loc[:, ['model', 'dataset', 'auc', 'f1_score', 'accuracy', 'precision']]

s1_result.head(5)

C:\Users\Bugi\AppData\Local\Temp\ipykernel_17068\975166161.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1_result.drop(columns=['scenario', 'loss', 'sensitivity'], inplace=True)
C:\Users\Bugi\AppData\Local\Temp\ipykernel_17068\975166161.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1_result.sort_values(by=['auc', 'f1_score', 'accuracy', 'precision'], ascending=False, inplace=True)


,model,dataset,auc,f1_score,accuracy,precision
14,mobilenet_v2,refuge,0.921856,0.965448,0.936667,0.949065
15,mobilenet_v2,rimone,0.893726,0.861264,0.812371,0.838678
13,mobilenet_v2,papila,0.732516,0.802653,0.709110,0.764606
10,efficientnet_v2s,refuge,0.691995,0.947785,0.900833,0.900753
7,efficientnet_v2m,rimone,0.669245,0.784461,0.645361,0.645361


In [17]:
# summarize only the model
s1_result_sumarize = s1_result[['model',
                                'auc',
                                'f1_score',
                                'accuracy',
                                'precision']].groupby(by='model').agg('mean')

s1_result_sumarize.reset_index(level=0, inplace=True)
s1_result_sumarize.sort_values(by=['auc', 'f1_score', 'accuracy', 'precision'], ascending=False, inplace=True)

s1_result_sumarize

,model,auc,f1_score,accuracy,precision
3,mobilenet_v2,0.754255,0.851743,0.777037,0.814453
4,mobilenet_v3large,0.569536,0.843095,0.733967,0.734342
2,efficientnet_v2s,0.565082,0.843135,0.735087,0.735919
0,efficientnet_v2l,0.564681,0.843326,0.734383,0.734383
1,efficientnet_v2m,0.562365,0.843326,0.734383,0.734383
5,mobilenet_v3small,0.528678,0.762229,0.697975,0.666179
